In [ ]:
#(1)Output Direct-Consumption-Coefficient-Matrix-A
import pandas as pd
import os

# Define list of years and input/output path templates
years = [2005, 2010, 2015, 2020]
input_template = r"D:\Desktop\Leontief-China\1-39x39-Input-Output-Matrices\{}.xlsx"
output_template = r"D:\Desktop\Leontief-China\2-Direct-Consumption-Matrices\Direct-Consumption-Coefficient-Matrix-A-{}.xlsx"

for year in years:
    # Construct file paths
    input_path = input_template.format(year)
    output_path = output_template.format(year)

    # Load Excel file
    df = pd.read_excel(input_path, skiprows=0, index_col=0)

    # Extract non-zero values from the last row
    last_values = df.iloc[-1, :]
    last_values = last_values[last_values != 0]

    # Perform division operation
    result_df = df.div(last_values, axis=1)

    # Remove the last row
    result_df = result_df.iloc[:-1, :]

    # Save results
    result_df.to_excel(output_path, sheet_name='Results')

    print(f'Results for {year} saved to: {output_path}')

print("Calculations completed for all years.")

In [ ]:
# (2)Output Leontief-Inverse-Matrices B
import pandas as pd
import numpy as np
import os

# Define the years and base input/output paths
years = [2005, 2010, 2015, 2020]
input_base_path = r"D:\Desktop\Leontief-China\2-Direct-Consumption-Matrices"
output_base_path = r"D:\Desktop\Leontief-China\3-Leontief-Inverse-Matrices"

# Ensure output directory exists
os.makedirs(output_base_path, exist_ok=True)

# Process each year
for year in years:
    # Construct input file path
    input_file = os.path.join(input_base_path, f"Direct-Consumption-Coefficient-Matrix-A-{year}.xlsx")
    
    # Load technical coefficient matrix A
    try:
        A = pd.read_excel(input_file, skiprows=0, index_col=0)
    except FileNotFoundError:
        print(f"Input file not found: {input_file}")
        continue
    except Exception as e:
        print(f"Error reading file {input_file}: {str(e)}")
        continue

    # Calculate identity matrix I
    I = np.eye(A.shape[0])

    # Calculate (I - A)
    I_minus_A = I - A.values

    # Calculate Leontief inverse matrix B
    try:
        B = np.linalg.inv(I_minus_A)
        
        # Convert B to DataFrame with same index and columns as A
        B_df = pd.DataFrame(B, index=A.index, columns=A.columns)
        
        # Construct output file path
        output_file = os.path.join(output_base_path, f"Leontief-Inverse-Matrix-B-{year}.xlsx")
        
        # Save the result
        B_df.to_excel(output_file)
        print(f"Leontief inverse matrix B for {year} saved to: {output_file}")
        
    except np.linalg.LinAlgError:
        print(f"Matrix (I - A) for {year} is singular and cannot be inverted.")
    except Exception as e:
        print(f"Error processing data for {year}: {str(e)}")

In [ ]:
#(3) Calculate Influence-and-Sensitivity Coefficients
import pandas as pd
import numpy as np
import os

# Define years and file paths
years = [2005, 2010, 2015, 2020]
input_base_path = r"D:\Desktop\Leontief-China\3-Leontief-Inverse-Matrices"
output_base_path = r"D:\Desktop\Leontief-China\4-Influence-and-Sensitivity-Coefficients"

# Ensure output directory exists
os.makedirs(output_base_path, exist_ok=True)

# Process each year
for year in years:
    # Construct input file path
    input_file = os.path.join(input_base_path, f"Leontief-Inverse-Matrix-B-{year}.xlsx")
    
    # Load Leontief inverse matrix B
    try:
        B = pd.read_excel(input_file, skiprows=0, index_col=0)
    except FileNotFoundError:
        print(f"Input file not found: {input_file}")
        continue
    except Exception as e:
        print(f"Error reading file {input_file}: {str(e)}")
        continue

    # Calculate Influence Coefficients
    # Step 1: Sum of each column
    column_sums = B.sum(axis=0)
    
    # Step 2: Average of column sums
    avg_column_sum = column_sums.mean()
    
    # Step 3: Influence coefficient = column sum / average
    influence_coefficients = column_sums / avg_column_sum

    # Calculate Sensitivity Coefficients
    # Step 1: Sum of each row
    row_sums = B.sum(axis=1)
    
    # Step 2: Average of row sums
    avg_row_sum = row_sums.mean()
    
    # Step 3: Sensitivity coefficient = row sum / average
    sensitivity_coefficients = row_sums / avg_row_sum

    # Create result DataFrame
    result_df = pd.DataFrame({
        'Sector_Name': B.index,
        'Influence_Coefficient': influence_coefficients.values,
        'Sensitivity_Coefficient': sensitivity_coefficients.values
    })

    # Construct output file path
    output_file = os.path.join(output_base_path, f"Influence-and-Sensitivity-Coefficients-{year}.xlsx")
    
    # Save results to Excel
    try:
        result_df.to_excel(output_file, index=False, sheet_name='Coefficients')
        print(f"Influence and sensitivity coefficients for {year} saved to: {output_file}")
    except Exception as e:
        print(f"Error saving results for {year}: {str(e)}")

print("Calculations completed for all years.")